<a href="https://colab.research.google.com/github/fedenemi/practice_university_of_michigan/blob/main/asign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

Using NetworkX, load in the bipartite graph from Employee_Movie_Choices.txt and return that graph.

In [ ]:
df = pd.read_csv("Employee_Movie_Choices.txt",delimiter="\t")
G=nx.from_pandas_dataframe(df,"#Employee","Movie")
    
plot_graph(G)  # Your Answer Here


Question 2
Using the graph from the previous question, add nodes attributes named 'type' where movies have the value 'movie' and employees have the value 'employee' and return that graph.

In [ ]:
def answer_two():
    

    df = pd.read_csv("Employee_Movie_Choices.txt",delimiter="\t")
    G=nx.from_pandas_dataframe(df,"#Employee","Movie")
    for a in list(employees):
        G.node[a]['type']="Employee"
        
    for a in list(movies):
        G.node[a]['type']="Movie"
    return  G#.nodes(data=True) #G2.nodes() # Your Answer Here

answer_two()

# Output: <networkx.classes.graph.Graph at 0x7f87f5edb400>

Find a weighted projection of the graph from answer_two which tells us how many movies different pairs of employees have in common.

In [ ]:
def answer_three():
        
    # Your Code Here
    G=answer_two()
    
    return bipartite.weighted_projected_graph(G,employees) # Your Answer Here

answer_three()

# Output: <networkx.classes.graph.Graph at 0x7f87f5e06cc0>

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using DataFrame.corr() ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation

In [ ]:
def answer_four():
    P = answer_three()
    
    rs = nx.read_edgelist('Employee_Relationships.txt', data=[('relationship_score', int)])
    rs= pd.DataFrame(rs.edges(data=True), columns=['a', 'b', 'relationship_score'])
    rs.relationship_score=rs.relationship_score.map(lambda x: x['relationship_score'] if type(x)==dict else None)    
    d=pd.DataFrame(P.edges(data=True), columns =['a1', 'a2', 'weight'])
    d['weight']=[list(v.values())[0] for k,v in d['weight'].items()]
    
    
    

    merged=pd.merge(rs, d, how='left', left_on=['a','b'], right_on=['a1','a2'])
    
    return merged.fillna(0)[["relationship_score","weight"]].corr().iloc[1,0]


answer_four()
# Output: 0.5233343182203728